In [129]:
# Dependencies
import pandas as pd
import datetime as dt
from datetime import timedelta
import numpy as np
import requests

In [130]:
Accidents_df = pd.read_csv("US_Accidents_Dec20_updated.csv")

In [131]:
pd.set_option('display.max_columns', None)

In [132]:
Accidents_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2716600,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,40.11206,-83.03187,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,NaN,Outerbelt E,R,Dublin,Franklin,OH,43017,US,US/Eastern,KOSU,2016-02-08 00:53:00,42.1,36.1,58.0,29.76,10.0,SW,10.4,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2716601,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,39.86501,-84.04873,0.747,At OH-4/OH-235/Exit 41 - Accident.,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
2,A-2716602,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,39.10209,-84.52396,0.055,At I-71/US-50/Exit 1 - Accident.,NaN,I-75 S,R,Cincinnati,Hamilton,OH,45203,US,US/Eastern,KLUK,2016-02-08 05:53:00,36.0,NaN,97.0,29.70,10.0,Calm,NaN,0.02,Overcast,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Day
3,A-2716603,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,39.09841,-84.52241,0.219,At I-71/US-50/Exit 1 - Accident.,NaN,US-50 E,R,Cincinnati,Hamilton,OH,45202,US,US/Eastern,KLUK,2016-02-08 05:53:00,36.0,NaN,97.0,29.70,10.0,Calm,NaN,0.02,Overcast,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Day
4,A-2716604,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,41.06217,-81.53547,0.123,At Dart Ave/Exit 21 - Accident.,NaN,I-77 N,R,Akron,Summit,OH,44311,US,US/Eastern,KAKR,2016-02-08 06:54:00,39.0,NaN,55.0,29.65,10.0,Calm,NaN,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Day,Day


In [133]:
Accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516064 entries, 0 to 1516063
Data columns (total 47 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID                     1516064 non-null  object 
 1   Severity               1516064 non-null  int64  
 2   Start_Time             1516064 non-null  object 
 3   End_Time               1516064 non-null  object 
 4   Start_Lat              1516064 non-null  float64
 5   Start_Lng              1516064 non-null  float64
 6   End_Lat                1516064 non-null  float64
 7   End_Lng                1516064 non-null  float64
 8   Distance(mi)           1516064 non-null  float64
 9   Description            1516064 non-null  object 
 10  Number                 469969 non-null   float64
 11  Street                 1516064 non-null  object 
 12  Side                   1516064 non-null  object 
 13  City                   1515981 non-null  object 
 14  County            

In [134]:
# Find the missing values
Accidents_df.isnull().sum()

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                        0
End_Lng                        0
Distance(mi)                   0
Description                    0
Number                   1046095
Street                         0
Side                           0
City                          83
County                         0
State                          0
Zipcode                      935
Country                        0
Timezone                    2302
Airport_Code                4248
Weather_Timestamp          30264
Temperature(F)             43033
Wind_Chill(F)             449316
Humidity(%)                45509
Pressure(in)               36274
Visibility(mi)             44211
Wind_Direction             41858
Wind_Speed(mph)           128862
Precipitation(in)         510549
Weather_Condition          44007
Amenity   

In [135]:
target_cols = ['ID','Severity','Start_Time','End_Time','Start_Lat','Start_Lng','City','State','Weather_Condition',
               'Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station',
              'Stop','Traffic_Calming','Traffic_Signal','Turning_Loop']


In [136]:
Accidents_df = Accidents_df[target_cols].copy()

In [137]:
Accidents_df['City'].value_counts()

Los Angeles       39984
Miami             36233
Charlotte         22203
Houston           20843
Dallas            19497
                  ...  
Upper Red Lake        1
Catoctin              1
Batesland             1
Laconia               1
Hubbardsville         1
Name: City, Length: 10657, dtype: int64

In [138]:
Accidents_df['State'].value_counts()

CA    448833
FL    153007
OR     87484
TX     75142
NY     60974
MN     52345
VA     51198
NC     50159
PA     42844
IL     41709
MD     35320
UT     33218
GA     31111
AZ     30185
NJ     29850
SC     28090
MI     27775
WA     22999
OH     22044
TN     21874
CO     19809
LA     19250
MO     16949
CT     15194
IN     11736
AL      9375
WI      7919
MA      6121
KY      5671
OK      5592
KS      5146
IA      4780
AR      4373
ID      4061
NH      3826
DC      3788
RI      3766
NV      3681
MT      3306
WV      3242
MS      2790
DE      2331
ME      2202
NE      2178
NM      1467
ND       455
VT       352
WY       330
SD       213
Name: State, dtype: int64

In [139]:
filt=(((Accidents_df['State']=='TX')&(Accidents_df['City']=='Dallas'))|
     ((Accidents_df['State']=='CA')&(Accidents_df['City']=='Los Angeles'))|
    ((Accidents_df['State']=='CA')&(Accidents_df['City']=='San Diego'))|
    ((Accidents_df['State']=='NY')&(Accidents_df['City']=='New York')))

Accidents_df=Accidents_df.loc[filt,:]

In [140]:
Accidents_df['City'].value_counts()

Los Angeles    39984
Dallas         19282
San Diego      14090
New York        4220
Name: City, dtype: int64

In [141]:
Accidents_df['StartDate'] = pd.to_datetime(Accidents_df['Start_Time'],errors='coerce').dt.date


In [142]:
Accidents_df['Year']=pd.to_datetime(Accidents_df['Start_Time'],errors='coerce').dt.year

In [143]:
Accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77576 entries, 5360 to 1516060
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 77576 non-null  object 
 1   Severity           77576 non-null  int64  
 2   Start_Time         77576 non-null  object 
 3   End_Time           77576 non-null  object 
 4   Start_Lat          77576 non-null  float64
 5   Start_Lng          77576 non-null  float64
 6   City               77576 non-null  object 
 7   State              77576 non-null  object 
 8   Weather_Condition  76876 non-null  object 
 9   Amenity            77576 non-null  bool   
 10  Bump               77576 non-null  bool   
 11  Crossing           77576 non-null  bool   
 12  Give_Way           77576 non-null  bool   
 13  Junction           77576 non-null  bool   
 14  No_Exit            77576 non-null  bool   
 15  Railway            77576 non-null  bool   
 16  Roundabout       

In [144]:
Accidents_df['Year'].value_counts()

2020    37787
2019    14903
2017     8926
2018     7996
2016     7964
Name: Year, dtype: int64

In [145]:
filt=((Accidents_df['Year']==2019)|
(Accidents_df['Year']==2020))
Accidents_df=Accidents_df.loc[filt,:]

In [146]:
Accidents_df['Year'].value_counts()

2020    37787
2019    14903
Name: Year, dtype: int64

In [147]:
Accidents_df['Start_Time'] = Accidents_df['Start_Time'].astype('datetime64')
Accidents_df['End_Time'] = Accidents_df['End_Time'].astype('datetime64')
Accidents_df['StartDate'] = Accidents_df['StartDate'].astype('datetime64')


In [148]:
Accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52690 entries, 237383 to 1516060
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID                 52690 non-null  object        
 1   Severity           52690 non-null  int64         
 2   Start_Time         52690 non-null  datetime64[ns]
 3   End_Time           52690 non-null  datetime64[ns]
 4   Start_Lat          52690 non-null  float64       
 5   Start_Lng          52690 non-null  float64       
 6   City               52690 non-null  object        
 7   State              52690 non-null  object        
 8   Weather_Condition  52137 non-null  object        
 9   Amenity            52690 non-null  bool          
 10  Bump               52690 non-null  bool          
 11  Crossing           52690 non-null  bool          
 12  Give_Way           52690 non-null  bool          
 13  Junction           52690 non-null  bool          
 14 

In [149]:
Accidents_df.drop(Accidents_df[Accidents_df['Year'] == 1900].index, inplace = True)
Accidents_df=Accidents_df.reset_index(drop=True)

In [150]:
Accidents_df.dropna(how = 'any',inplace=True)


In [151]:
Accidents_df=Accidents_df.sort_values('StartDate').reset_index(drop=True)

In [152]:
Accidents_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,City,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,StartDate,Year
0,A-3918803,3,2019-01-01 21:04:45,2019-01-01 21:34:27,32.85392,-96.64543,Dallas,TX,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,2019-01-01,2019
1,A-3918991,3,2019-01-01 15:13:46,2019-01-01 15:43:03,34.06092,-118.25032,Los Angeles,CA,Clear,False,False,False,False,True,False,False,False,False,False,False,False,False,2019-01-01,2019
2,A-3918984,2,2019-01-01 13:04:00,2019-01-01 13:33:59,34.04965,-118.44876,Los Angeles,CA,Clear,False,False,False,False,True,False,False,False,False,False,False,False,False,2019-01-01,2019
3,A-3918983,2,2019-01-01 12:54:15,2019-01-01 13:23:45,34.06996,-118.26170,Los Angeles,CA,Clear,False,False,False,False,False,False,False,False,False,False,False,False,False,2019-01-01,2019
4,A-3918776,3,2019-01-01 02:06:15,2019-01-01 02:36:06,32.87535,-96.77010,Dallas,TX,Clear,False,False,False,False,True,False,False,False,False,False,False,False,False,2019-01-01,2019


In [153]:
Accidents_df.to_csv("cleaned_dataV1.csv",index=False)